In [190]:
%run '00_rh_settings.ipynb'

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['norm', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [191]:
#data_GA = data.query("Stock == 'GOOG' | Stock == 'AAPL'").dropna().groupby("Stock")["Return"]
data_AFG = data.pivot_table(index='Date', columns='Stock', values='Return')[["AAPL","FB","GOOG"]].dropna()
data_AFG.head()

Stock,AAPL,FB,GOOG
Date,,,
2014-03-28,0.001118,0.015997,-0.002732
2014-03-31,0.000224,-0.003818,0.005422
2014-04-01,-0.009065,-0.038007,-0.017967
2014-04-02,-0.001659,-0.001594,0.000282
2014-04-03,0.006979,0.054295,-0.004809


In [192]:
#data.query("Stock == 'GOOG' | Stock == 'AAPL' | Stock == 'FB'")[["Date","Stock","Return"]].dropna().groupby(["Date","Stock"], as_index=False).head()

## Granger causality

In [199]:
statsmodels.api.tsa.stattools.grangercausalitytests(data_AFG[['GOOG', 'AAPL']], maxlag=10, addconst=True);


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1173  , p=0.7320  , df_denom=942, df_num=1
ssr based chi2 test:   chi2=0.1177  , p=0.7315  , df=1
likelihood ratio test: chi2=0.1177  , p=0.7315  , df=1
parameter F test:         F=0.1173  , p=0.7320  , df_denom=942, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1321  , p=0.8762  , df_denom=939, df_num=2
ssr based chi2 test:   chi2=0.2657  , p=0.8756  , df=2
likelihood ratio test: chi2=0.2657  , p=0.8756  , df=2
parameter F test:         F=0.1321  , p=0.8762  , df_denom=939, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.2936  , p=0.8300  , df_denom=936, df_num=3
ssr based chi2 test:   chi2=0.8875  , p=0.8285  , df=3
likelihood ratio test: chi2=0.8870  , p=0.8286  , df=3
parameter F test:         F=0.2936  , p=0.8300  , df_denom=936, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.2131  , p=0.9312  

__Вывод:__ у акциий Apple и Google нет значения лага, при которой они в значительной мере линейно зависимы

## Считать корреляцию плохая идея, поэтому мы будем использовать пакет который пытались портировать из R

In [332]:
stats.pearsonr(data_AFG.AAPL, data_AFG.GOOG)

(0.007287643720310545, 0.8228713018425238)

В пакете перенесли не все функции, он "сырой", но это хотя бы что-то.

Оригинальный пакет из R: https://cran.r-project.org/web/packages/egcm/index.html

Его порт на Python: https://github.com/David-OConnor/egcm

In [201]:
from egcm.test import *
from egcm import *

Given two time series $Y[t]$ and $X[t]$, the Engle Granger cointegration
model in its simplest form looks for alpha, beta and rho such that

$Y[t] = \alpha + \beta * X[t] + R[t]$

$R[t] = \rho * R[t-1] + \epsilon[t]$

where $epsilon[t]$ is a series of independent and identically distributed
innovations with mean zero. If $\alpha$, $\beta$ and $\rho$ can be found such that
$-1 < \rho < 1$, then the series are said to be cointegrated. If $abs(\rho) = 1$,
then the residual series $R[t]$ is said to have a unit root (or alternatively,
it is said to follow a random walk).

Похоже, порт не совсем корректно считает P-value, поэтому используем дополнительную проверку.

In [333]:
def cointegration_check(egcm_result):
    # Check for cointegration without pvalue
    keys = ['alpha', 'beta','rho']
    parameters = [egcm_result.get(key) for key in keys]
    df = pd.DataFrame({'keys': keys, 'parameters': parameters})
    return df, all(abs(df.parameters) < 1)

### Тестовые данные

In [334]:
norm_values = scipy.stats.norm.rvs(size=100)

test_data0 = pd.DataFrame({"First":norm_values, "Second":200*norm_values+200})
test_data1 = pd.DataFrame({"First":norm_values, "Second":scipy.stats.norm.rvs(size=100)})
test_data2 = pd.DataFrame({"First":norm_values, "Second":norm_values+norm_values})
test_data3 = pd.DataFrame({"First":norm_values, "Second":range(0,100)+sin(range(0,100))+norm_values+norm_values*norm_values})
test_data4 = pd.DataFrame({"First":norm_values, "Second":range(0,100)})
test_data = test_data1

test_egcm_result = egcm(test_data["First"], test_data["Second"], robust=True)
test_egcm(test_egcm_result)
is_cointegrated(test_egcm_result)
cointegration_check(test_egcm_result)

{'statistic': -107.41003214767116,
 'alternative': 'cointegrated',
 'p_value': 0.0001,
 'method': 'Engle Granger two-step cointegration test pp',
 'urtest': 'Phillips-Perron test',
 'data_name': 'Placeholder'}

False

(    keys  parameters
 0  alpha    0.054603
 1   beta   -0.135373
 2    rho   -0.075385, True)

## Apple и Google

In [335]:
#import egcm
egcm_result = egcm(X=data_AFG.AAPL, Y=data_AFG.GOOG, robust=True)

# egcm(X,Y) -- Constructs an Engle-Granger cointegration model from X & Y
# summary.egcm(E) -- Prints various summary statistics on the Engle-Granger
# cointegration model constructed from X & Y
# plot.egcm(E) -- Creates a graph of the Engle-Granger cointegration model
test_egcm(egcm_result)
is_cointegrated(egcm_result)
cointegration_check(egcm_result)

{'statistic': -820.5217778380179,
 'alternative': 'cointegrated',
 'p_value': 9.999999999999999e-05,
 'method': 'Engle Granger two-step cointegration test pp',
 'urtest': 'Phillips-Perron test',
 'data_name': 'Placeholder'}

False

(    keys  parameters
 0  alpha   -0.000697
 1   beta   -0.000288
 2    rho    0.038026, True)

Могут быть связаны